In [22]:
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from gensim import corpora
import gensim
import pandas as pd
from MyTop import Top2Vec
import re
from textblob import TextBlob
import contractions    
from numpy import inf

# DATA LOAD

In [23]:
data1 = pd.read_csv('[Billboard]Top_5000_Songs_with_Translations.csv')
data2 = pd.read_csv('[멜론]Top_4500_Songs_with_Translations.csv')

In [269]:
Pop_artist = data1['Artist']

# KPOP 한글 전처리

In [24]:
from konlpy.tag import Okt
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MJ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\MJ\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [25]:
def clean_text(text):
    content = text
    cleaned_text = re.sub(r'[^가-힣a-zA-Z0-9 ]', '', content) # 한글 / 영어 알파벳과 숫자(0~9)와 공백(1개)까지만 포함
    cleaned_text = re.sub(r'([오])\1+', r'\1', cleaned_text)  # '오' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([루])\1+', r'\1', cleaned_text)  # '오' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([르])\1+', r'\1', cleaned_text)  # '오' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([어])\1+', r'\1', cleaned_text)  # '오' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([베])\1+', r'\1', cleaned_text)  # '오' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([라])\1+', r'\1', cleaned_text)  # '라' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([워])\1+', r'\1', cleaned_text)  # '워' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([나])\1+', r'\1', cleaned_text)  # '오' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([척])\1+', r'\1', cleaned_text)  # '오' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([우])\1+', r'\1', cleaned_text)  # '우' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'([하])\1+', r'\1', cleaned_text)  # '하' 란 글자 2개 이상 이면 제거함
    cleaned_text = re.sub(r'(la)\1+', r'\1', cleaned_text)    # 'la' 란 글자(들)이 2개 이상 이면 제거함
    cleaned_text = re.sub(r'(두근)\1+', r'\1', cleaned_text)  # '두근' 란 글자(들)이 2개 이상 이면 제거함
    cleaned_text = re.sub(r"(\w)\1+(\w)\2+", r"\1\2", cleaned_text)
    cleaned_text = re.sub(r'(.)\1{3,}', r'\1', cleaned_text)  #  그외 하나의 글자(또는 Character)가 3개이상 일 때는 제거함
    #cleaned_text = re.sub(r'(.+?)\1+', r'\1', cleaned_text)  # stemming 또는 Hello 같은 'm'자 또는 'l'자도 하나로만 사용하고 싶을 때 사용해도됨.
    return cleaned_text

In [26]:
kr_stopwords ='아 휴 아이구 에르 쿠안 코모 에스 포르 비엔 페로 야야 하소 야만 마다나 어머나 순간 오안 하라 가야 오난 타요 래야 티엔 엘라 프레 안도 데스 아호 아시 하스타 파라 올비 건초 라나 마디 다야 타요 마오 요나 아누 래야 야오 야만 자기야 페라리 마넌 니키 무언가 다누 중이 다나 마미 아아 다오 게토 야야 오지 마내 똑바로 하나요 마이바흐 도아 건가 다날 해당 상이 아난 내내 여지 오네가 가득 바비 테라 마스 라스 모스 로스 데일 라마 스토 리오 스타 디아 멜로 프리 가나 제이 에나 주사위 미스 다드 아모 프로 아미 스트로 칼리 부스 보라 리아 레고 카르 크리 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 이제 고통 모든 가장 위해 대가 고우리 때그 왜냐면 갑자기 그게 이유 이건 야우리 깜둥이 항상 거나 젠장 정말 려고 타고 예전 지그 째깍 다그 오늘이 로나 번의 해나 난나 다가 라리 어쩌 그거 빙빙 놨다 어머 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 하니 하자놓 듯이 라면 거기 여기 때내 오라 라서 뚜루 한단 요사 엄머머머 잠시 아예 에이'
kr_stopwords = kr_stopwords.split()

In [27]:
okt = Okt()
kpop_kr =[]
for i in tqdm(range(4577)):
    add=[]
    text = data2['Lyrics'][i]
    cleaned_text = clean_text(text)
    nouns = list(okt.nouns(cleaned_text))
    lemmatized_words = [word for word in nouns if len(word) >= 2]
    stopwords_removed_sentence = [word for word in lemmatized_words if not word in kr_stopwords]
    for word in stopwords_removed_sentence:
        docs = "".join(word)
        add.append(docs)
    doc = " ".join(add)
    kpop_kr.append(doc)

100%|██████████████████████████████████████████████████████████████████████████████| 4577/4577 [02:29<00:00, 30.52it/s]


In [41]:
df_kpop_kr=pd.DataFrame(kpop_kr)

# POP 한글 전처리 및 Top2vec

In [34]:
okt = Okt()
pop_kr =[]
for i in tqdm(range(5147)):
    add=[]
    text = data1['Translations'][i]
    cleaned_text = clean_text(text)
    nouns = list(okt.nouns(cleaned_text))
    lemmatized_words = [word for word in nouns if len(word) >= 2]
    stopwords_removed_sentence = [word for word in lemmatized_words if not word in kr_stopwords]
    for word in stopwords_removed_sentence:
        docs = "".join(word)
        add.append(docs)
    doc = " ".join(add)
    pop_kr.append(doc)

100%|██████████████████████████████████████████████████████████████████████████████| 5147/5147 [05:48<00:00, 14.76it/s]


In [44]:
df_pop_kr=pd.DataFrame(pop_kr)

# POP 영어 전처리 및 Top2vec

In [4]:
def complete_words(text):
    # 1.단계 --> 철자 체크
    textBlb = TextBlob(text)
    fixed_sentence = textBlb.correct()
    f_fixed_sentence = str(fixed_sentence)
    
    # 2. 단계 --> 짧은 단어들을 풀어주는 역할
    expanded_words = []
    for word in f_fixed_sentence.split():
        expanded_words.append(contractions.fix(word))   
    
    expanded_text = ' '.join(expanded_words)
    return str(f_fixed_sentence)

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MJ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stopwords=stopwords.words('english')
newstopwords= ['oh', 'ooh', 'eh', 'oooh', 'whoa', 'im', 'mmh','boom','whataya', 'nobody', 'tiempo', 'pero', 'otra', 'estoy', 'nunca', 'ahora', 'porque', 'hace', 'siempre', 'cuando', 'ella', 'vamo', 'bien', 'nadie', 'conmigo', 'como', 'tiene', 'esta', 'vida', 'quiero', 'noche', 'sola', 'contigo', 'puedo', 'hasta', 'amor', 'gusta', 'tengo', 'para', 'todo', 'dale', 'loca', 'mami', 'dura', 'rico', 'dice', 'dime', 'flow', 'lambo', 'hookah', 'gate', 'solo', 'chris', 'papi', 'size', 'pill', 'real', 'blunt' 'imma', 'ohh', 'uh', 'la', 'li', 'im', 'na', 'huh', 'ah', 'ou', 'oou', 'yeah', 'de', 'p']
stopwords_list=stopwords+newstopwords

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
wlem = nltk.WordNetLemmatizer()
pop_en = []
for i in tqdm(range(5147)):
    add = []
    whole_text = data1['Lyrics'][i]
    whole_text = re.sub('[^a-zA-Z ]+',' ',whole_text)
    no_capitals = whole_text.lower().split()
    no_stops = [word for word in no_capitals if not word in stopwords_list]
    
    lem_word = [wlem.lemmatize(w) for w in no_stops]
    len_word = [word for word in lem_word if len(word) >= 4]
    len_word = [word for word in len_word if len(word) <= 12]
    tokens_pos = nltk.pos_tag(len_word)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos:
            NN_words.append(word)
        if 'JJ' in pos:
            NN_words.append(word)

    words = ' '.join(NN_words)
    completed_words = complete_words(words)
    
    for word in completed_words:

        words3 = "".join(word)
        add.append(words3)
    doc = "".join(add)
    pop_en.append(doc)

100%|████████████████████████████████████████████████████████████████████████████| 5147/5147 [1:34:04<00:00,  1.10s/it]


In [45]:
df_pop_en=pd.DataFrame(pop_en)

# Kpop 영어 전처리 및 Top2vec

In [56]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
wlem = nltk.WordNetLemmatizer()
kpop_en = []
for i in tqdm(range(4577)):
    add = []
    whole_text = data2['Filtered'][i]
    whole_text = re.sub('[^a-zA-Z ]+',' ',whole_text)
    no_capitals = whole_text.lower().split()
    no_stops = [word for word in no_capitals if not word in stopwords_list]
    
    lem_word = [wlem.lemmatize(w) for w in no_stops]
    len_word = [word for word in lem_word if len(word) >= 4]
    len_word = [word for word in len_word if len(word) <= 12]
    tokens_pos = nltk.pos_tag(len_word)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos:
            NN_words.append(word)
        if 'JJ' in pos:
            NN_words.append(word)

    words = ' '.join(NN_words)
    completed_words = complete_words(words)
    
    for word in completed_words:

        words3 = "".join(word)
        add.append(words3)
    doc = "".join(add)
    kpop_en.append(doc)

 89%|█████████████████████████████████████████████████████████████████████▎        | 4577/5147 [33:34<04:10,  2.27it/s]


KeyError: 4577

In [67]:
kpop_en

['weird heart face imagine control facial expression crazy miss anna first want fingers mouth control heart broken heart point pull love baby you talk everyone whistle reduce everyone cut voice call open love first fingers mouth control heart broken heart point pull love baby long time heart love word control heart broken heart point pull love baby hello hello meet talk melon think',
 'nice smile jealous make afraid give trust jealous make thinking phone feel sleep night moonlight confess feel feel look heart anxious hold late here love song cool everyday love dance spend night look heart anxious hold late hehehehe wrong bring push pull honest jealous make curious jealous make love everyday know feel everyday everyday know feel',
 'heart refuge someone black tunnel invisible path desert thirsty person kiss baby night hurry miss guess much night pass silver moon scent memory dream rain kiss baby love kiss baby tear ddurururu ddurudu wind ddurururu ddurudu draw daddy night miss memory he

In [59]:
df_kpop_kr=pd.DataFrame(kpop_kr)
df_pop_kr=pd.DataFrame(pop_kr)
df_pop_en=pd.DataFrame(pop_en)
df_kpop_en=pd.DataFrame(kpop_en)

In [60]:
pop_pre=pd.concat([df_pop_kr, df_pop_en], axis=1)
kpop_pre=pd.concat([df_kpop_kr, df_kpop_en], axis=1)

In [61]:
pop_pre.columns = ['pop_kr_Lyrics', 'pop_en_Lyrics']
kpop_pre.columns = ['kpop_kr_Lyrics', 'kpop_en_Lyrics']

In [62]:
pop_pre.to_csv('pop_pre.csv', index=False, encoding='utf-8-sig')
kpop_pre.to_csv('kpop_pre.csv', index=False, encoding='utf-8-sig')

# Kpop 한글 Top2vec

In [72]:
kpop_kr_model = Top2Vec(kpop_kr)

2021-12-14 03:33:23,599 - top2vec - INFO - Pre-processing documents for training
2021-12-14 03:33:23,983 - top2vec - INFO - Creating joint document/word embedding
2021-12-14 03:33:30,706 - top2vec - INFO - Creating lower dimension embedding of documents
2021-12-14 03:33:35,072 - top2vec - INFO - Finding dense areas of documents
2021-12-14 03:33:35,254 - top2vec - INFO - Finding topics


In [73]:
kpop_kr_topic_words, kpop_kr_word_scores, kpop_kr_topic_nums = kpop_kr_model.get_topics()
len(kpop_kr_topic_nums)

28

In [202]:
kpop_kr_topic_list = []
for i in range(len(kpop_kr_topic_nums)):
    kpop_kr_topic_list.append(kpop_kr_topic_words[i][:10].tolist())
kpop_kr_topic_list

[['필요', '손짓', '손끝', '큰일', '천사', '눈앞', '거부', '수록', '공간', '게이'],
 ['수배', '현상', '집착', '대화', '침대', '나인', '이불', '지네', '굳이', '일도'],
 ['행복', '너머', '공간', '건너', '파도', '푸른', '우릴', '두운', '태양', '기도'],
 ['솔로', '연애', '영혼', '용기', '지구', '수배', '너뿐이야', '현상', '관심', '우주'],
 ['파란', '눈동자', '글자', '천국', '온기', '비밀', '말투', '송이', '미소', '분명'],
 ['웃기', '미안', '모해', '오해', '이해', '취한', '감당', '꿈속', '박수', '반대'],
 ['소린', '믿음', '역사', '마마', '바닥', '현실', '보이지', '리가', '미로', '주기'],
 ['외톨이야', '도록', '번지', '굿바이', '대론', '부디', '유리', '편지', '사람과', '온몸'],
 ['지난날', '글자', '감동', '스물', '봄바람', '비교', '냄새', '금은', '존재', '크리스마스'],
 ['꿈속', '악몽', '지도', '의미', '갈수록', '일지', '어젯밤', '진실', '관심', '금은'],
 ['허리', '무릎', '어쩜', '관심', '별로', '이불', '전화번호', '전화기', '머리', '굳이'],
 ['주르륵', '자격', '소린', '입맞춤', '피하', '입가', '거릴', '어우리', '자고', '수도'],
 ['현상', '수배', '금도', '우연', '서서', '마주', '안녕', '결혼', '밤하늘', '우주'],
 ['분명', '제자리', '피해', '의심', '연기', '공기', '지도', '가기', '감고', '존재'],
 ['외로움', '하필', '자격', '다해', '보이지', '힘껏', '두운', '어둠', '믿음', '습관'],
 ['첫사랑', '뿌리', '는걸', '의심', '일지

# Pop 한글 Top2vec

In [75]:
pop_kr_model = Top2Vec(pop_kr)

2021-12-14 03:33:35,326 - top2vec - INFO - Pre-processing documents for training
2021-12-14 03:33:36,210 - top2vec - INFO - Creating joint document/word embedding
2021-12-14 03:33:47,044 - top2vec - INFO - Creating lower dimension embedding of documents
2021-12-14 03:33:51,224 - top2vec - INFO - Finding dense areas of documents
2021-12-14 03:33:51,460 - top2vec - INFO - Finding topics


In [76]:
pop_kr_topic_words, pop_kr_word_scores, pop_kr_topic_nums = pop_kr_model.get_topics()
len(pop_kr_topic_nums)

46

In [83]:
pop_kr_topic_list = []
for i in range(len(pop_kr_topic_nums)):
    pop_kr_topic_list.append(pop_kr_topic_words[i][:10].tolist())
pop_kr_topic_list

[['글록', '계집애', '변호사', '막대기', '리처드', '저격수', '헬기', '방금', '참호', '동생'],
 ['안고', '세월', '희망이', '낙원', '사람과', '연인', '낯선', '기쁨', '미소', '후회'],
 ['감히', '전설', '동의', '괴물', '배신', '고모', '빚진', '평화', '오마이갓', '최근'],
 ['구걸', '문화', '불평', '개인', '교훈', '싱글', '축복', '책임', '목적', '마음'],
 ['시골', '컨트리', '뒷길', '라디오', '마을', '타이어', '맥주', '트럭', '맨발', '먼지'],
 ['예수님', '주님', '아버지', '믿음', '신의', '평화', '하나님', '희망', '하느님', '상처'],
 ['불길', '자제력', '한밤중', '자연', '세월', '상처', '가장자리', '오늘밤', '어쩌면', '운명'],
 ['위스키', '테네시', '맥주', '햇살', '여름', '시골', '뒷길', '남쪽', '마을', '라디오'],
 ['영감', '뉴욕', '정글', '번가', '유행', '거리', '건배', '말리', '택시', '나이'],
 ['굿나잇', '해제', '소금', '앞길', '젊은이', '습관', '실현', '분노', '멀리', '구석'],
 ['이상은', '다라', '위험', '이자', '절망', '그림자', '낯선', '감수', '샤워', '세월'],
 ['작별', '손길', '인사', '작별인사', '미소', '가끔', '갈망', '눈빛', '안고', '굿나잇'],
 ['외로워', '부정', '작별', '연인', '어쩌면', '손길', '작별인사', '슬픔', '안나', '절망'],
 ['행복', '시절', '인간', '유행', '세트', '며칠', '배합', '운영', '세계', '거절'],
 ['천둥', '빗속', '다비', '번개', '자연', '죄악', '동물', '눈물', '구름', '불꽃'],
 ['무감각', '해지', '러브'

# Pop 영어 Top2vec

In [78]:
pop_en_model = Top2Vec(pop_en)

2021-12-14 03:33:51,549 - top2vec - INFO - Pre-processing documents for training
2021-12-14 03:33:52,567 - top2vec - INFO - Creating joint document/word embedding
2021-12-14 03:34:13,387 - top2vec - INFO - Creating lower dimension embedding of documents
2021-12-14 03:34:17,863 - top2vec - INFO - Finding dense areas of documents
2021-12-14 03:34:18,076 - top2vec - INFO - Finding topics


In [79]:
pop_en_topic_words, pop_en_word_scores, pop_en_topic_nums = pop_en_model.get_topics()
len(pop_en_topic_nums)

34

In [80]:
pop_en_topic_list = []
for i in range(len(pop_en_topic_nums)):
    pop_en_topic_list.append(pop_en_topic_words[i][:10].tolist())
pop_en_topic_list

[['trench',
  'bitch',
  'clip',
  'block',
  'tint',
  'shit',
  'business',
  'rapier',
  'clock',
  'richard'],
 ['country',
  'beer',
  'radio',
  'cowboy',
  'southern',
  'dust',
  'whiskey',
  'truck',
  'dirt',
  'town'],
 ['survive',
  'strength',
  'ground',
  'alive',
  'tear',
  'fire',
  'fade',
  'pride',
  'world',
  'fear'],
 ['deserve',
  'beautiful',
  'perfect',
  'somebody',
  'mistake',
  'love',
  'book',
  'worst',
  'someone',
  'anybody'],
 ['jesus',
  'father',
  'lord',
  'daughter',
  'mother',
  'heal',
  'faith',
  'piece',
  'prison',
  'throne'],
 ['melody',
  'sing',
  'rhythm',
  'song',
  'radio',
  'singing',
  'dance',
  'music',
  'note',
  'moon'],
 ['came',
  'est',
  'eyes',
  'also',
  'same',
  'die',
  'beside',
  'pierre',
  'sent',
  'more'],
 ['numb',
  'telephone',
  'calling',
  'emotion',
  'pain',
  'mission',
  'left',
  'busy',
  'relate',
  'claim'],
 ['awake',
  'wide',
  'inside',
  'mistake',
  'daylight',
  'cloud',
  'fear',
  

# Kpop 영어 Top2vec

In [63]:
kpop_en_model = Top2Vec(kpop_en)

2021-12-14 03:30:36,655 - top2vec - INFO - Pre-processing documents for training
2021-12-14 03:30:37,233 - top2vec - INFO - Creating joint document/word embedding
2021-12-14 03:30:45,331 - top2vec - INFO - Creating lower dimension embedding of documents
2021-12-14 03:30:49,078 - top2vec - INFO - Finding dense areas of documents
2021-12-14 03:30:49,297 - top2vec - INFO - Finding topics


In [64]:
kpop_en_topic_words, kpop_en_word_scores, kpop_en_topic_nums = kpop_en_model.get_topics()
len(pop_en_topic_nums)

48

In [236]:
kpop_en_topic_list = []
for i in range(len(kpop_en_topic_nums)):
    kpop_en_topic_list.append(kpop_en_topic_words[i][:10].tolist())
kpop_en_topic_list

[['money',
  'success',
  'rapier',
  'level',
  'plan',
  'neck',
  'stage',
  'trend',
  'rich',
  'best'],
 ['rainy',
  'umbrella',
  'longing',
  'forgotten',
  'rain',
  'memory',
  'breakup',
  'painful',
  'window',
  'cloudy'],
 ['shake',
  'rhythm',
  'bounce',
  'raise',
  'dance',
  'floor',
  'party',
  'body',
  'everybody',
  'volume'],
 ['foolish',
  'fault',
  'cried',
  'sorry',
  'loved',
  'several',
  'tear',
  'heal',
  'knew',
  'fool'],
 ['shadow',
  'breakup',
  'jumping',
  'dark',
  'sigh',
  'darkness',
  'pain',
  'memory',
  'tear',
  'weak'],
 ['forgot',
  'disease',
  'loved',
  'scar',
  'hate',
  'hurt',
  'able',
  'painful',
  'breakup',
  'flurry'],
 ['blossom',
  'spring',
  'cherry',
  'petal',
  'breeze',
  'bloom',
  'resembles',
  'leaf',
  'season',
  'wind'],
 ['phone',
  'hang',
  'kind',
  'relationship',
  'attention',
  'text',
  'mouth',
  'honest',
  'worse',
  'busy'],
 ['foolish',
  'painful',
  'ache',
  'breakup',
  'cried',
  'forgo

In [152]:
len(kpop_kr_topic_nums)

28

In [153]:
len(pop_kr_topic_nums)

46

In [154]:
len(kpop_en_topic_nums)

50

In [155]:
len(pop_en_topic_nums)

34

# KPOP_KR_PWI

In [222]:
count=0
for i in tqdm(range(4577)):
    a =len(kpop_kr[i])
    count = count+a
count

100%|█████████████████████████████████████████████████████████████████████████| 4577/4577 [00:00<00:00, 1529789.58it/s]


587852

In [224]:
from numpy import inf

kpop_kr_total_pwi=[]
N = 4577

F = count

P_dj = (1 / N)

for i in tqdm(range(len(kpop_kr_topic_nums))):
    dj_wi_pwi_list = []
    for d in pop_kr:
        words_in_t = []
        #Iterate over n topic words
        fake_topic_words = kpop_kr_topic_list[i]
        for word in fake_topic_words:
            tokenized_doc = [x.lower() for x in list(d.split(' '))] 
            f_ij = sum([1 for x in tokenized_doc if word == x])
            P_wi_dj = (f_ij / F)
            P_wi = freq_dist.freq(word)
            try:
                if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
                    pmi = 0
                else :
                    pmi = np.log(P_wi_dj/(P_wi * P_dj))
            except:
                pass
            pwi = P_wi_dj * pmi
            words_in_t.append(pwi)
            dj_wi_pwi_list.append(pwi)
    topic_pwi = sum(dj_wi_pwi_list)
    print(topic_pwi)
    kpop_kr_total_pwi.append(topic_pwi)

  0%|                                                                                           | 0/28 [00:00<?, ?it/s]<ipython-input-224-794fc3adce1d>:22: RuntimeWarning: divide by zero encountered in log
  if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
  4%|██▉                                                                                | 1/28 [00:00<00:24,  1.09it/s]

0.009212113214213216


  7%|█████▉                                                                             | 2/28 [00:01<00:24,  1.07it/s]

0.0068661748591183425


 11%|████████▉                                                                          | 3/28 [00:02<00:23,  1.07it/s]

...

0.0049862600043099605


 96%|███████████████████████████████████████████████████████████████████████████████   | 27/28 [00:24<00:00,  1.09it/s]

0.0033272396642122573


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:25<00:00,  1.08it/s]

0.014462301120783394


In [229]:
sum(kpop_kr_total_pwi)

0.26228614971353864

In [205]:
import pandas as pd

Topic_Pwi = pd.DataFrame()
Topic_Pwi['keywords']= kpop_kr_topic_list
Topic_Pwi['pwi']=total_pwi

In [206]:
Topic_Pwi['keywords'][0]

['필요', '손짓', '손끝', '큰일', '천사', '눈앞', '거부', '수록', '공간', '게이']

In [209]:
Topic_Pwi_sort_value = Topic_Pwi.sort_values('pwi', ascending=False)
Topic_Pwi_sort_value

,keywords,pwi
18,"[거울, 제일, 낯선, 보지, 아마, 번만, 버릇, 수만, 마나, 음악]",0.003184
12,"[현상, 수배, 금도, 우연, 서서, 마주, 안녕, 결혼, 밤하늘, 우주]",0.002832
21,"[조심, 주인공, 여름, 뭔가, 분위기, 공기, 조명, 무대, 세계, 게이]",0.002666
19,"[아저씨, 온몸, 아주, 무릎, 키스, 어서, 물어, 며칠, 가기, 어깨]",0.002535
2,"[행복, 너머, 공간, 건너, 파도, 푸른, 우릴, 두운, 태양, 기도]",0.002477
23,"[먼지, 나야, 서울, 달리, 발자국, 불빛, 피어, 구름, 높이, 불어]",0.002460
25,"[외톨이야, 모해, 굿바이, 커피, 박수, 주르륵, 아내, 전화번호, 부디, 그대만]",0.002412
26,"[커피, 어쩜, 꿈속, 마다, 술잔, 자꾸, 나로, 귓가, 한잔, 색깔]",0.002363
27,"[완전, 장난, 분위기, 거지, 영화, 뭔가, 연인, 방법, 벌써, 감동]",0.002362
20,"[표현, 설레임, 필요, 말로, 도난, 일도, 걸너, 영화, 변명, 눈앞]",0.002354


# Pop_kr_PWI

In [226]:
from nltk import ngrams, FreqDist
freq_dist = FreqDist()
for doc in pop_kr:
    doc = doc.split()
    for word in doc:
        freq_dist[word.lower()] += 1
freq_dist

FreqDist({'사랑': 10793, '우린': 4537, '그냥': 3569, '그래그래': 2780, '마음': 2476, '친구': 2360, '오늘': 2269, '계속': 2220, '절대': 2131, '노래': 1876, ...})

In [227]:
count=0
for i in tqdm(range(5147)):
    a =len(pop_kr[i])
    count = count+a
count

100%|█████████████████████████████████████████████████████████████████████████| 5147/5147 [00:00<00:00, 2581997.69it/s]


1357885

In [228]:
from numpy import inf

pop_kr_total_pwi=[]
N = 5147

F = count

P_dj = (1 / N)

for i in tqdm(range(len(pop_kr_topic_nums))):
    dj_wi_pwi_list = []
    for d in pop_kr:
        words_in_t = []
        #Iterate over n topic words
        fake_topic_words = pop_kr_topic_list[i]
        for word in fake_topic_words:
            tokenized_doc = [x.lower() for x in list(d.split(' '))] 
            f_ij = sum([1 for x in tokenized_doc if word == x])
            P_wi_dj = (f_ij / F)
            P_wi = freq_dist.freq(word)
            try:
                if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
                    pmi = 0
                else :
                    pmi = np.log(P_wi_dj/(P_wi * P_dj))
            except:
                pass
            pwi = P_wi_dj * pmi
            words_in_t.append(pwi)
            dj_wi_pwi_list.append(pwi)
    topic_pwi = sum(dj_wi_pwi_list)
    print(topic_pwi)
    pop_kr_total_pwi.append(topic_pwi)

  0%|                                                                                           | 0/46 [00:00<?, ?it/s]<ipython-input-228-02029e237824>:22: RuntimeWarning: divide by zero encountered in log
  if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
  2%|█▊                                                                                 | 1/46 [00:00<00:41,  1.09it/s]

0.0043567993808159445


  4%|███▌                                                                               | 2/46 [00:01<00:40,  1.09it/s]

0.0027989983448495413


  7%|█████▍                                                                             | 3/46 [00:02<00:39,  1.09it/s]

...

0.0033725080753944665


 98%|████████████████████████████████████████████████████████████████████████████████▏ | 45/46 [00:41<00:00,  1.09it/s]

0.0035536565720197797


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:42<00:00,  1.09it/s]

0.0033513189695350606


In [230]:
sum(pop_kr_total_pwi)

0.1604042077277694

# Kpop_en_Pwi

In [232]:
from nltk import ngrams, FreqDist
freq_dist = FreqDist()
for doc in kpop_en:
    doc = doc.split()
    for word in doc:
        freq_dist[word.lower()] += 1
freq_dist

FreqDist({'love': 7930, 'heart': 7796, 'time': 6638, 'night': 3741, 'baby': 3531, 'everything': 3095, 'good': 2681, 'girl': 2629, 'look': 2304, 'today': 2175, ...})

In [234]:
count=0
for i in tqdm(range(4577)):
    a =len(kpop_en[i])
    count = count+a
count

100%|█████████████████████████████████████████████████████████████████████████| 4577/4577 [00:00<00:00, 2189476.44it/s]


1939883

In [240]:
from numpy import inf

kpop_en_total_pwi=[]
N = 4577

F = count

P_dj = (1 / N)

for i in tqdm(range(len(kpop_en_topic_nums))):
    dj_wi_pwi_list = []
    for d in kpop_en:
        words_in_t = []
        #Iterate over n topic words
        fake_topic_words = kpop_en_topic_list[i]
        for word in fake_topic_words:
            tokenized_doc = [x.lower() for x in list(d.split(' '))] 
            f_ij = sum([1 for x in tokenized_doc if word == x])
            P_wi_dj = (f_ij / F)
            P_wi = freq_dist.freq(word)
            try:
                if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
                    pmi = 0
                else :
                    pmi = np.log(P_wi_dj/(P_wi * P_dj))
            except:
                pass
            pwi = P_wi_dj * pmi
            words_in_t.append(pwi)
            dj_wi_pwi_list.append(pwi)
    topic_pwi = sum(dj_wi_pwi_list)
    print(topic_pwi)
    kpop_en_total_pwi.append(topic_pwi)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]<ipython-input-240-d2c4c7f59763>:22: RuntimeWarning: divide by zero encountered in log
  if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
  2%|█▋                                                                                 | 1/50 [00:00<00:28,  1.74it/s]

0.001617495068082791


  4%|███▎                                                                               | 2/50 [00:01<00:27,  1.74it/s]

0.0018074945392969482


  6%|████▉                                                                              | 3/50 [00:01<00:27,  1.74it/s]

...

0.0019931364196203493


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [00:28<00:00,  1.73it/s]

0.0023383711172481176


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:29<00:00,  1.72it/s]

0.001697863983203284


In [241]:
sum(kpop_en_total_pwi)

0.08925311327965542

# Pop_en_PWI

In [242]:
from nltk import ngrams, FreqDist
freq_dist = FreqDist()
for doc in pop_en:
    doc = doc.split()
    for word in doc:
        freq_dist[word.lower()] += 1
freq_dist

FreqDist({'night': 11938, 'baby': 7931, 'bitch': 6815, 'time': 6648, 'cause': 6443, 'girl': 6252, 'love': 5666, 'anna': 5196, 'life': 3650, 'fuck': 3562, ...})

In [243]:
count=0
for i in tqdm(range(5147)):
    a =len(pop_en[i])
    count = count+a
count

100%|█████████████████████████████████████████████████████████████████████████| 5147/5147 [00:00<00:00, 2580146.13it/s]


3489346

In [244]:
from numpy import inf

pop_en_total_pwi=[]
N = 5147

F = count

P_dj = (1 / N)

for i in tqdm(range(len(pop_en_topic_nums))):
    dj_wi_pwi_list = []
    for d in pop_en:
        words_in_t = []
        #Iterate over n topic words
        fake_topic_words = pop_en_topic_list[i]
        for word in fake_topic_words:
            tokenized_doc = [x.lower() for x in list(d.split(' '))] 
            f_ij = sum([1 for x in tokenized_doc if word == x])
            P_wi_dj = (f_ij / F)
            P_wi = freq_dist.freq(word)
            try:
                if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
                    pmi = 0
                else :
                    pmi = np.log(P_wi_dj/(P_wi * P_dj))
            except:
                pass
            pwi = P_wi_dj * pmi
            words_in_t.append(pwi)
            dj_wi_pwi_list.append(pwi)
    topic_pwi = sum(dj_wi_pwi_list)
    print(topic_pwi)
    pop_en_total_pwi.append(topic_pwi)

  0%|                                                                                           | 0/34 [00:00<?, ?it/s]<ipython-input-244-d6593d561403>:22: RuntimeWarning: divide by zero encountered in log
  if np.log(P_wi_dj/(P_wi * P_dj)) == -inf:
  3%|██▍                                                                                | 1/34 [00:00<00:30,  1.08it/s]

0.0008826859555534895


  6%|████▉                                                                              | 2/34 [00:01<00:29,  1.07it/s]

0.0015368276400689468


  9%|███████▎                                                                           | 3/34 [00:02<00:29,  1.06it/s]

...

0.000550563869834772


 97%|███████████████████████████████████████████████████████████████████████████████▌  | 33/34 [00:30<00:00,  1.08it/s]

0.0011356557438076545


100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:31<00:00,  1.07it/s]

0.0013284728285838177


In [245]:
sum(pop_en_total_pwi)

0.03507539451522584

# Top2vec 문서추천 한국어

In [247]:
kpop_kr_word_list=[]
for i in range(10):
    a = kpop_kr_topic_list[i][0]
    kpop_kr_word_list.append(a)

In [248]:
kpop_kr_word_list

['필요', '수배', '행복', '솔로', '파란', '웃기', '소린', '외톨이야', '지난날', '꿈속']

,0
0,수도 우주 열기구 공기 상관 지붕 손뼉 행복 진리 박수 행복 박수 소식 고난 경고 ...
1,할리 사이 휘발유 판매 군대 부인 문신 요흥 방송국 노래 정부 수도 수백 전형 행복...
2,여보 나무 세월 은신 음악 불신 재창 직면 행복 행복 행복 타박상 지나 저주 절규 ...
3,호호 행복 행복 행복 크리스마스 이야기 세상 기억 조립 옛날 옛적 마을 소녀 소원 ...
4,뭔가 기회 통제 불능 그냥 바보 게임 얼굴 보기 행복 심장 시작 나이 사진 액자 절...


In [285]:
documents, document_scores, document_ids = pop_kr_model.search_documents_by_keywords(keywords=['맥주'], num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print('Title : ', Pop_title[int(doc_id)], ', Artist : ', Pop_artist[int(doc_id)])
    print("-----------")
    print()

Document: 3109, Score: 0.5868451595306396
-----------
Title :  ONE BEER , Artist :  HARDY Feat. Lauren Alaina & Devin Dawson
-----------

Document: 3579, Score: 0.5364699363708496
-----------
Title :  Cold Beer Calling My Name , Artist :  Jameson Rodgers & Luke Combs
-----------

Document: 3294, Score: 0.5269269943237305
-----------
Title :  Beers And Sunshine , Artist :  Darius Rucker
-----------

Document: 4115, Score: 0.5105506777763367
-----------
Title :  Cold One , Artist :  Eric Church
-----------

Document: 3778, Score: 0.5041806697845459
-----------
Title :  Shark In The Water  , Artist :  V V Brown
-----------



In [264]:
Pop_title[4508]

'Happy Does  '

'  \n87년산 할리를 몰다\n오전 8시에서 3시 사이에 휘발유와 마시는 것을 판매합니다\n군대와 전 부인 4번 문신이 있어요\n흥민이 따라다니면서 자기 방송국에 있는 노래들을 널 가르쳤지\n그는 정부를 욕하는 또 다른 사람이 될 수도 있다\n아니\n하지만 그는 웃는다\n안 될 이유는 수백 가지지만\n그는 그의 전형이다\n행복은 행복만큼 중요하다\n6번 트랙을 잡고\n밧줄 그네를 찾아라\n트럭에 야자수 매달아라\n그냥 맥주나 마셔요\n비바람 속에서 느린 춤을 훔쳐라\n그리고 당신이 사랑하는 사람으로부터의 키스\n반 컵 가득히 먹고 웃으며 살아라\n그래\n행복은 행복한 것만큼\n어떤 이는 성경이나 폴라로이드 그림에서 찾을 수 있다\n동전 던지던지 앞면 던지던지 애리조나 투손으로 가는거야\n하지만 네가 하지 않았던 일들을 돌이켜보진 않을 거야\n가끔은 그렇게 느끼겠지만\n삶이 더 나아질 때\n행복은 행복만큼 중요하다\n여섯 줄을 잡고\n밧줄 그네를 찾아라\n트럭에 야자수 매달아라\n그냥 맥주나 마셔요\n비바람 속에서 느린 춤을 훔쳐라\n그리고 당신이 사랑하는 사람으로부터의 키스\n반 컵 가득히 먹고 웃으며 살아라\n행복은 행복한 것만큼 좋은 거니까 그게 우리가 원하는 전부야\n그게 다야\n우리가 원하는 전부야\n그녀는 예전의 그녀와는 거리가 멀다\n볼더에 있는 이 산에서 다이빙을 합니다\n모든 피클 항아리\n구겨진 달러\n1마일 더 높이 올라가서\n행복은 행복만큼 중요하다\n여섯 줄을 잡고\n밧줄 그네를 찾아라\n트럭에 야자수 매달아라\n그냥 맥주나 마셔요\n비바람 속에서 느린 춤을 훔쳐라\n그리고 당신이 사랑하는 사람으로부터의 키스\n반 컵 가득히 먹고 웃으며 살아라\n먹구름이 잔뜩 끼면 태양을 찾아라\n행복은 행복한 것만큼 좋은 거니까 그게 우리가 원하는 전부야\n그게 다야\n우리가 원하는 전부야\n행복은 행복한 것만큼 좋은 것이다\n그게 다야\n반 컵 가득히 먹고 웃으며 살아라'